In [1]:
import numpy as np
import pandas as pd
import tables
from tqdm import tqdm 

/opt/conda/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/conda/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
dt = pd.read_csv('/home/leyla/data/train_5-6.csv')

In [3]:
def save_chunk(name, data, labels, mask):
    h5file = tables.open_file("/home/leyla/data/hd5_3/{}.h5".format(name), mode="w", filters=tables.Filters(complevel=4))
    
    h5file.create_carray("/", "data", obj=data)
    h5file.create_carray("/", "labels", obj=labels)
    h5file.create_carray("/", "mask", obj=mask)

    
    h5file.close()

In [4]:
E_SCALE = 50
energy_cut=0.01

In [5]:
batch_size = 5

data = np.zeros((batch_size, 192, 192, 192))
labels = np.zeros((batch_size, 192, 192, 192), dtype='byte')
mask = np.zeros((batch_size, 192, 192, 192), dtype='byte')
offset = 0


for _, row in tqdm(dt.iterrows(), total=dt.shape[0]):
    event = int(row['event'])
    i = event - offset
    if (i >= batch_size):
        save_chunk("train_" + str(offset // batch_size).zfill(4), data, labels, mask)
        
        data = np.zeros((batch_size, 192, 192, 192))
        labels = np.zeros((batch_size, 192, 192, 192), dtype='byte')
        mask = np.zeros((batch_size, 192, 192, 192), dtype='byte')
        
        offset += batch_size
        i = 0
        
    label = int(row['label'])
    x = int(row['x']) 
    y = int(row['y'])
    z = int(row['z'])
    if row['val'] > energy_cut:
        data[i, x, y, z] = row['val'] / E_SCALE
    labels[i, x, y, z] = label
    mask[i, x, y, z] = 1

save_chunk("train_" + str(offset // batch_size).zfill(4), data, labels, mask)

100%|██████████| 12591689/12591689 [1:26:04<00:00, 2438.25it/s]


In [7]:
dt = pd.read_csv('/home/leyla/data/test_5-6.csv')

In [8]:
def save_chunk(name, data, mask):
    h5file = tables.open_file("/home/leyla/data/hd5_3/{}.h5".format(name), mode="w", filters=tables.Filters(complevel=4))
    
    h5file.create_carray("/", "data", obj=data)    
    h5file.create_carray("/", "mask", obj=mask)

    
    h5file.close()

In [9]:
batch_size = 5

data = np.zeros((batch_size, 192, 192, 192))
mask = np.zeros((batch_size, 192, 192, 192), dtype='byte')
offset = 0


for _, row in tqdm(dt.iterrows(), total=dt.shape[0]):
    event = int(row['event'])
    i = event - offset
    if (i >= batch_size):
        save_chunk("test_" + str(offset // batch_size).zfill(4), data, mask)
        
        data = np.zeros((batch_size, 192, 192, 192))
        labels = np.zeros((batch_size, 192, 192, 192), dtype='byte')
        mask = np.zeros((batch_size, 192, 192, 192), dtype='byte')
        
        offset += batch_size
        i = 0
            
    x = int(row['x']) 
    y = int(row['y'])
    z = int(row['z'])
    data[i, x, y, z] = row['val']
    labels[i, x, y, z] = label
    mask[i, x, y, z] = 1

save_chunk("test_" + str(offset // batch_size).zfill(4), data, mask)

100%|██████████| 12502922/12502922 [2:19:49<00:00, 1490.29it/s] 
